read ae expanded & annotate network

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, sys
from tqdm import tqdm
plt.style.use('default')

# load ae_net source code
sys.path.append('/mnt/disk7t/xwj/axolotl_rev/gitcode')
from functions import *

load packaged functions

In [2]:
! which python

/mnt/disk7t/xwj/soft/miniforge3/envs/py3.12/bin/python


In [3]:
# output directory
outdir = '/mnt/disk7t/xwj/axolotl_rev/output' 

os.makedirs(outdir+'/data', exist_ok=True)
os.makedirs(outdir+'/sweet', exist_ok=True)

filename_nd_template = '{outdir}/data/node_df_{sample}.txt'
filename_ed_template = '{outdir}/data/edge_df_{sample}.txt'
filename_ed_aberrant_template = '{outdir}/data/edge_df_{sample}_aberrant.txt'
filename_scatter_template = '{outdir}/data/edge_scatter_{sample}.pdf'

reading and processing different datasets related to gene expression and outlier detection.

In [4]:
# path to files from upstream AE detection tools
datadir = '/mnt/disk7t/xwj/axolotl_rev/pfib_423/'

## example score of pfib423 dataset
result_baseline = {
    'outsingle':f'{datadir}/osg_res_1_001.txt',
    'outrider': f'{datadir}/outrider_res_1_001.txt',
    'outrider_cts': f'{datadir}/outrider_norm_cts_1_001.txt',
    'trueoutlier': f'{datadir}/df_true49.txt',
}
# your_dataset_file = '/media/eys/xwj/axolotl_rev/pfib_423/df_cts_gm2022_dedup.txt'

#------------------------------------------------------------------------#
# 原文提供了全部样本的详细注释信息，可以用来对数据集进行去批次效应
file = f'{datadir}/sample_annot.tsv'
s_anno = pd.read_csv(file, sep="\t", index_col='RNA_ID')
# pfib数据集包含两种测序平台strand & non strand，two groups 表现为两个批次. 分别保存
# rename: stranded -> ss； non-stranded -> ns
s_anno['group'] = s_anno['STRAND_SPECIFIC'].map({True:'ss', False:'ns'})
s_anno.describe()

#------------------------------------------------------------------------#
'''上游异常表达基因识别工具的输出结果, 此处假设有2种方法的结果,分别为OUTSINGLE和OUTRIDER'''

# method 1. 
file = result_baseline['outsingle']
score = pd.read_csv(file, sep="\t", index_col=0)

# method 2.
# if use OUTRIDER res
file = result_baseline['outrider']
score = pd.read_csv(file, sep="\t", index_col=0)
score = score.pivot_table(values='pValue',index= 'geneID', columns='sampleID')

'''not used'''
cts_outrider = pd.read_csv(result_baseline['outrider_cts'], sep="\t", index_col=0)

#------------------------------------------------------------------------#
# 根据原文附件，我整理了这组样本，每个样本有candidate异常基因, 作为数据集的label
# file = f'{datadir}/df_true3.txt'
# file = f'{datadir}/df_true49.txt'
# outlier = pd.read_csv(file, sep="\t", index_col=None)
outlier = pd.read_csv(result_baseline['trueoutlier'], sep="\t", index_col=None)
outlier['group'] = s_anno.loc[ outlier['Sample'],'group'].values

#------------------------------------------------------------------------#
'''表达量矩阵，作为共表达相关性依据，用于寻找异常表达基因之间的最短路径'''
file = f'{datadir}/t0_g12543_s423.txt.gz'
cts = pd.read_csv(file, sep="\t", index_col=0)
exp = cts.transform(lambda x: np.log(x+1))


In [5]:
outlier.head()

,Sample,Sex,Gene,RNAdefects,TableName,group
0,R80184,M,ALDH18A1,"AE, MAE",S2,ns
1,R60537,M,ATP6AP1,"AE, Var",S2,ss
2,R62943,F,MICOS13,"AE, AS",S2,ss
3,R96820,F,CLPP,"AE, AS",S2,ns
4,R77611,F,DLD,"AE, MAE",S2,ss


In [6]:
s_anno.head()

,INDIVIDUAL_ID,TISSUE,SEX,AFFECTED,ICD_10,PAIRED_END,STRAND_SPECIFIC,group
RNA_ID,,,,,,,,
R0052,p11688,Fibroblast,Female,True,E88,True,False,ns
R0053,p48517,Fibroblast,Male,True,E88,True,False,ns
R19907,p46808,Fibroblast,Male,True,E88,True,False,ns
R10581,p51250,Fibroblast,Female,True,E88,True,False,ns
R0055,p5413,Fibroblast,Female,True,E88,True,False,ns


In [7]:
# dataset split by sequencing library type (stranded or non-stranded) as batches
batches = ['ss','ns']

# filename dict
dict_filename = dict.fromkeys(batches)

for group in batches:
    
    # expression matrix 
    sample_list = s_anno.query('group == @group').index.tolist()
    expfile = f'{outdir}/df_exp_{group}{len(sample_list)}.txt'
    exp.loc[:, sample_list].to_csv( expfile, sep='\t')
    
    # score matrix
    scorefile = f'{outdir}/df_score_{group}{len(sample_list)}.txt'
    score.loc[:, sample_list].to_csv( scorefile, sep='\t')
    
    # sample meta table
    sample_list_interest = outlier.query('group == @group')['Sample']
    outlierfile = f'{outdir}/df_true_{group}{len(sample_list_interest)}.txt'
    outlier.query('Sample in @sample_list_interest').to_csv(outlierfile,sep='\t')
        
    dict_filename[group] = {}
    dict_filename[group]['exp'] = expfile
    dict_filename[group]['score'] = scorefile
    dict_filename[group]['outlier'] = outlierfile
    print(group, len(sample_list), len(sample_list_interest))
    
print(dict_filename)

ss 269 25
ns 154 24
{'ss': {'exp': '/mnt/disk7t/xwj/axolotl_rev/output/df_exp_ss269.txt', 'score': '/mnt/disk7t/xwj/axolotl_rev/output/df_score_ss269.txt', 'outlier': '/mnt/disk7t/xwj/axolotl_rev/output/df_true_ss25.txt'}, 'ns': {'exp': '/mnt/disk7t/xwj/axolotl_rev/output/df_exp_ns154.txt', 'score': '/mnt/disk7t/xwj/axolotl_rev/output/df_score_ns154.txt', 'outlier': '/mnt/disk7t/xwj/axolotl_rev/output/df_true_ns24.txt'}}


### run aberrant network and edge analysis

In [ ]:
%%time
start_time = datetime.datetime.now()

thres_score = 0.0001 # threshold of ae scores
thres_q_pcc = 0.01 # in percent threshold of whole pearsons matrix 
ae_gene_max = 20  # num of top ae genes 
pcc_gene_max = 10  # num of correlated genes per ae gene; affect network size significantly
pcc_gene_min = 5 # if not have any correlated genes, take some genes below threshold.

# ------------------------------
# Part 1. Dataset Batch & Sample Selection 
# ------------------------------
# cts = No normalize
# log = log transform
# use log(cts) or log(cpm) compute Pearsons' correlation as global coexpression network edges.
# expression table using raw counts log transformed (Optional also do count per million normalization)
c, ctype, p = 'raw', 'cts', 'log' # user can leave it as default

# s_list = [ 'R52016','R28774'] #from ns sample
# s_list = [ "R15264" ] #from ss sample

# load data
group = 'ss'
file_e = dict_filename[group]['exp']
df_exp = pd.read_csv(file_e, sep="\t", index_col=0)

file_sc = dict_filename[group]['score']
df_score = pd.read_csv(file_sc, sep="\t", index_col=0)

outlierfile = dict_filename[group]['outlier']
df_outlier = pd.read_csv(outlierfile, sep="\t", index_col=0)

# select subset of samples
if True:
    s_list = df_outlier['Sample'].tolist()

log_time(start_time, "load expression data...")

#-----------------------------------------------------------
# prepare intermediate data
#-----------------------------------------------------------

# correlation
pcc = pd.DataFrame( np.corrcoef(df_exp), index=df_exp.index.tolist(), columns=df_exp.index.tolist())
pcc.index.name, pcc.columns.name ='gene1','gene2'

log_time(start_time, "Prepare coexpression pcc matrix")

# pcc remove duplicate, get undirected network = 0.5 min
# pcc = dict_pcc[c][ctype][p].copy()
# pcc_tri = pd.Series(pcc.stack()).reset_index().rename(columns={0:'pcc'})
# pcc_tri = pcc_tri[ pcc_tri['gene1'] < pcc_tri['gene2'] ]

pcc_tri = pearson_extract_upper_triangle(pcc)
log_time(start_time, "Prepare coexpression edge table")

print(f"{'DataFrame':<15} | {'Shape':<10}")
print("-" * 40)
print(f"{'df_outlier':<15} | {df_outlier.shape}")
print(f"{'df_exp':<15} | {df_exp.shape}")
print(f"{'df_score':<15} | {df_score.shape}")
print(f"{'pcc':<15} | {pcc.shape}")
print(f"{'pcc_tri':<15} | {pcc_tri.shape}")
print("\n")

# input data: df_exp, df_score, df_outlier, pcc_tri, pcc
kwargs = {'output_dir': outdir,
          'df_exp': df_exp, # df
          'df_score': df_score, # df
          'df_outlier': df_outlier, # df
          'pcc_tri': pcc_tri,# df
          'pcc': pcc, # df
          'c': c, 
          'ctype': ctype,
          'p': p,
          'thres_score': thres_score, 
          'thres_q_pcc': thres_q_pcc,
          'pcc_gene_min': pcc_gene_min, 
          'pcc_gene_max': pcc_gene_max, 
          'ae_gene_max': ae_gene_max,
          'file_e': file_e, # sweet need this file path to compute sample weight
         }

# ------------------------------
# Part 1. construct Reference Network for datasets and all samples( ~ 0.5min )
# ------------------------------
G_int = global_net(**kwargs)
log_time(start_time, "Global coexpress network of all genes")
kwargs['G_int'] = G_int     # Update kwargs

STEP: load expression data...
Current Time: 2025-05-18 22:06:07 Elapsed Time: 0:00:01.130880

STEP: Prepare coexpression pcc matrix
Current Time: 2025-05-18 22:06:10 Elapsed Time: 0:00:03.360219

STEP: Prepare coexpression edge table
Current Time: 2025-05-18 22:06:14 Elapsed Time: 0:00:08.021681

DataFrame       | Shape     
----------------------------------------
df_outlier      | (25, 6)
df_exp          | (12543, 269)
df_score        | (12543, 269)
pcc             | (12543, 12543)
pcc_tri         | (78657153, 3)


Nodes:8447     Edges:786572   Is Connected:False

Nodes:12543    Edges:828197   Is Connected:True

STEP: Global coexpress network of all genes
Current Time: 2025-05-18 22:06:41 Elapsed Time: 0:00:35.210311

CPU times: user 49.8 s, sys: 4.16 s, total: 53.9 s
Wall time: 35.2 s


In [ ]:
# ------------------------------
# Part 2. Generate AE Network for One Sample
# ------------------------------


for s in tqdm(s_list):
    
    log_time(start_time, f"{s} AE network start")
    
    nodes, edges = ae_net(s, **kwargs)
    nodes.to_csv(filename_nd_template.format(outdir=outdir, sample=s), sep='\t', header=True, index=True)
    edges.to_csv(filename_ed_template.format(outdir=outdir, sample=s), sep='\t', header=True, index=True)
    
    # identifying aberrant edges
    edges = label_aberrant_edges(edges)
    edges.to_csv(filename_ed_aberrant_template.format(outdir=outdir, sample=s), sep='\t', header=True, index=True)
    # scatter plots to show aberrant edges
    fig, axs = create_scatter_plot(edges)
    fig.savefig(filename_scatter_template.format(outdir=outdir, sample=s ), bbox_inches='tight')
    plt.close(fig)
    
    log_time(start_time, f"{s} AE network finish")

  0%|                                                                                                                                                                                  | 0/25 [00:00<?, ?it/s]

STEP: R60537 AE network start
Current Time: 2025-05-18 22:23:26 Elapsed Time: 0:17:20.202269

AE net: get AE genes
Gene       | AE Value   | Edge      
----------------------------------------
ATP6AP1    | 3.82e-40   | 7    
SRI        | 6.02e-11   | 10   
TP53I3     | 3.35e-09   | 5    
ARMC10     | 1.56e-06   | 10   
DAP3       | 2.53e-05   | 10   
MRPL3      | 6.54e-05   | 10   
MCM3       | 6.78e-05   | 10   
SYNM       | 7.58e-05   | 5    
CUTALP     | 8.48e-05   | 5    
RPS6       | 9.08e-05   | 10   


AE net: check AE network connectivity, 1
Nodes:92       Edges:82       Is Connected:False

Modules: [8, 11, 6, 11, 11, 11, 11, 6, 6, 11]
AE net: shortest paths between modules
 prior
1.0    73
0.5    14
2.0    11
Name: count, dtype: int64
AE net: check AE network connectivity, 2
Nodes:158      Edges:246      Is Connected:True

AE net: run SWEET analysis
patients:{'R60537'}, index:[252], total samples:269, total genes:12543



  4%|██████▊                                                                                                                                                                   | 1/25 [00:30<12:18, 30.78s/it]

STEP: R60537 AE network finish
Current Time: 2025-05-18 22:23:57 Elapsed Time: 0:17:50.980084

STEP: R62943 AE network start
Current Time: 2025-05-18 22:23:57 Elapsed Time: 0:17:50.982713

AE net: get AE genes
Gene       | AE Value   | Edge      
----------------------------------------
MICOS13    | 3.50e-19   | 10   
ATIC       | 7.81e-17   | 10   
SUOX       | 3.20e-12   | 6    
PXDC1      | 7.29e-06   | 5    
SEPTIN2    | 1.95e-05   | 10   
BMS1P5     | 4.45e-05   | 5    
KHDRBS3    | 9.80e-05   | 5    


AE net: check AE network connectivity, 1
Nodes:58       Edges:51       Is Connected:False

Modules: [11, 11, 7, 6, 11, 6, 6]
AE net: shortest paths between modules
 prior
1.0    35
0.5     7
2.0     2
Name: count, dtype: int64
AE net: check AE network connectivity, 2
Nodes:109      Edges:148      Is Connected:True

AE net: run SWEET analysis
patients:{'R62943'}, index:[14], total samples:269, total genes:12543



  8%|█████████████▌                                                                                                                                                            | 2/25 [00:58<11:11, 29.19s/it]

STEP: R62943 AE network finish
Current Time: 2025-05-18 22:24:25 Elapsed Time: 0:18:19.059665

STEP: R77611 AE network start
Current Time: 2025-05-18 22:24:25 Elapsed Time: 0:18:19.061050

AE net: get AE genes
Gene       | AE Value   | Edge      
----------------------------------------
DLD        | 5.75e-30   | 10   
PUM3       | 1.20e-16   | 10   
DDX24      | 2.17e-09   | 10   
PIK3C2A    | 1.72e-08   | 10   
NRBP2      | 2.18e-08   | 5    
B3GALNT2   | 4.03e-08   | 10   
DOCK9      | 1.38e-07   | 5    
UGGT2      | 4.73e-06   | 10   
RBL1       | 1.16e-05   | 10   
KRT34      | 2.51e-05   | 5    
SRP72      | 3.76e-05   | 10   
DPH6       | 5.16e-05   | 10   
PSMG3-AS1  | 7.01e-05   | 5    
NUBPL      | 7.90e-05   | 10   


AE net: check AE network connectivity, 1
Nodes:129      Edges:120      Is Connected:False

Modules: [39, 11, 11, 11, 6, 11, 6, 11, 11, 6, 6]
AE net: shortest paths between modules
 prior
1.0    135
0.5     17
2.0     12
Name: count, dtype: int64
AE net: check AE

In [ ]:
# ! which python


In [12]:
# label aberrant edges

In [ ]:
    # plot edge attributes and labels
    # n_row, n_col = 1, 3
    # fig, axs = plt.subplots(n_row, n_col, sharey=False,figsize=(6*n_col, 6*n_row))
    # plt.subplots_adjust( wspace=0.2, hspace=0.2)
    # sns.scatterplot(data=edges, x = 'residual_z', y = 'outscore', hue='group', style='label', size='abs(sweet-pcc)', alpha=0.8, ax=axs[0])
    # sns.scatterplot(data=edges, x = 'residual_z', y = 'abs(sweet-pcc)', hue='group', style='label', size='outscore', alpha=0.8,ax=axs[1])
    # sns.scatterplot(data=edges, x = 'abs(sweet-pcc)', y = 'outscore', hue='group', style='label', alpha=0.8, ax=axs[2])
    # # idx = 3
    # # sns.scatterplot(data=edges, x = 'residual_z', y = 'abs(sin-pcc)', hue='group', size='outscore', alpha=0.8,ax=axs[idx])
    # plt.savefig(filename_scatter_template.format(outdir=datadir, sample=s ), bbox_inches='tight')
    # plt.close(fig)

In [51]:
edges

,gene1,gene2,group,sweet,pcc,outscore,residual_z,abs(sweet-pcc),label
0,TANGO2,DCTN1,AE,0.4646,0.838919,3.533572,-7.046113,0.374319,aberrant
1,TANGO2,TMEM127,AE,0.2751,0.836316,3.838883,-7.901387,0.561170,aberrant
2,TANGO2,TBC1D13,AE,0.4954,0.832722,3.442074,-6.774214,0.337360,aberrant
3,TANGO2,WIPI2,AE,0.4563,0.830722,3.707770,-6.936302,0.374423,aberrant
4,TANGO2,RAB5C,AE,0.4500,0.827429,3.755403,-6.909761,0.377478,aberrant
...,...,...,...,...,...,...,...,...,...
2208,TOX4,FAF2,ADD2,0.8755,0.901236,1.647343,1.731929,0.025747,normal
2209,PBXIP1,BBS1,ADD2,0.8696,0.869819,1.011160,-0.136087,0.000190,normal
2210,PBXIP1,MEGF8,ADD2,0.8706,0.870489,1.040907,0.121240,0.000116,normal
2211,CCM2,PPP1CA,ADD2,0.9040,0.898310,1.020430,-0.002923,0.005498,normal
